In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
#  Mini API Client

import requests
import json

# ================================
#  APPELER UNE API PUBLIQUE
# ================================

def appeler_api(url):
    """Envoie une requête GET à l'URL et retourne les données JSON"""
    try:
        print(f" Connexion à l'API...")
        reponse = requests.get(url)                  # envoi de la requête

        if reponse.status_code == 200:               # 200 = succès
            print(f" Réponse reçue (code {reponse.status_code})")
            return reponse.json()                    # convertit JSON → dict Python
        else:
            print(f" Erreur API (code {reponse.status_code})")
            return None

    except requests.exceptions.ConnectionError:
        print(" Pas de connexion internet !")
        return None


# ================================
#  CONVERTIR JSON → TABLEAU
# ================================

def json_vers_tableau(donnees, colonnes):
    """Convertit une liste de dicts JSON en tableau lisible"""
    tableau = []

    for item in donnees:
        ligne = {}
        for col in colonnes:
            ligne[col] = item.get(col, "N/A")        # .get() évite les KeyError
        tableau.append(ligne)

    return tableau


def afficher_tableau(tableau, titre=""):
    """Affiche les données sous forme de tableau formaté"""
    if not tableau:
        print("Aucune donnée à afficher.")
        return

    if titre:
        print(f"\n{'=' * 55}")
        print(f"  {titre}")
        print(f"{'=' * 55}")

    colonnes = list(tableau[0].keys())

    # En-tête
    entete = "  ".join(f"{col:<18}" for col in colonnes)
    print(f"\n  {entete}")
    print("  " + "-" * (20 * len(colonnes)))

    # Lignes
    for ligne in tableau:
        valeurs = "  ".join(f"{str(ligne[col]):<18}" for col in colonnes)
        print(f"  {valeurs}")


# ================================
#  ANALYSES SIMPLES
# ================================

def analyser_donnees(tableau, colonne_numerique):
    """Calcule des stats basiques sur une colonne numérique"""
    valeurs = []

    for ligne in tableau:
        try:
            valeurs.append(float(ligne[colonne_numerique]))
        except (ValueError, TypeError):
            pass                                     # on ignore les valeurs non numériques

    if not valeurs:
        return

    print(f"\n STATISTIQUES — colonne '{colonne_numerique}' :")
    print(f"  • Nombre   : {len(valeurs)}")
    print(f"  • Minimum  : {min(valeurs):.2f}")
    print(f"  • Maximum  : {max(valeurs):.2f}")
    print(f"  • Moyenne  : {sum(valeurs)/len(valeurs):.2f}")


# ================================
#  PROGRAMME PRINCIPAL
# ================================

print("=" * 55)
print("   MINI API CLIENT")
print("=" * 55)
print("  APIs disponibles :")
print("  1 → Pokémon  (pokémons et leurs stats)")
print("  2 → Pays     (informations sur les pays)")
print("  3 → Blagues  (blagues aléatoires en anglais)")
print("=" * 55)

choix = input("Votre choix : ").strip()

# ── API 1 : PokéAPI ──────────────────────────────────────
if choix == "1":
    nb = input("Combien de pokémons afficher ? (ex: 10) : ").strip()
    url = f"https://pokeapi.co/api/v2/pokemon?limit={nb}"

    donnees = appeler_api(url)

    if donnees:
        pokemons = []
        for i, p in enumerate(donnees["results"], start=1):
            pokemons.append({"#": i, "nom": p["name"], "url": p["url"]})

        afficher_tableau(pokemons, titre=f" {nb} PREMIERS POKÉMONS")
        print(f"\n  Total disponibles : {donnees['count']}")

# ── API 2 : REST Countries ───────────────────────────────
elif choix == "2":
    region = input("Région (europe / asia / africa / americas) : ").strip()
    url    = f"https://restcountries.com/v3.1/region/{region}"

    donnees = appeler_api(url)

    if donnees:
        pays_list = []
        for p in donnees:
            pays_list.append({
                "pays"       : p.get("name", {}).get("common", "N/A"),
                "capitale"   : p.get("capital", ["N/A"])[0] if p.get("capital") else "N/A",
                "population" : p.get("population", 0)
            })

        # Tri par population décroissante
        pays_list = sorted(pays_list, key=lambda x: x["population"], reverse=True)

        afficher_tableau(pays_list[:10], titre=f" TOP 10 PAYS — {region.upper()}")
        analyser_donnees(pays_list, "population")

# ── API 3 : JokeAPI ──────────────────────────────────────
elif choix == "3":
    url     = "https://v2.jokeapi.dev/joke/Any?type=single&amount=5"
    donnees = appeler_api(url)

    if donnees:
        print(f"\n 5 BLAGUES ALÉATOIRES")
        print("=" * 55)
        for i, blague in enumerate(donnees["jokes"], start=1):
            print(f"\n  {i}. {blague['joke']}")
            print(f"  Catégorie : {blague['category']}")
        print("=" * 55)

else:
    print(" Choix invalide !")


   MINI API CLIENT
  APIs disponibles :
  1 → Pokémon  (pokémons et leurs stats)
  2 → Pays     (informations sur les pays)
  3 → Blagues  (blagues aléatoires en anglais)


Votre choix :  1
Combien de pokémons afficher ? (ex: 10) :  5


 Connexion à l'API...
 Réponse reçue (code 200)

   5 PREMIERS POKÉMONS

  #                   nom                 url               
  ------------------------------------------------------------
  1                   bulbasaur           https://pokeapi.co/api/v2/pokemon/1/
  2                   ivysaur             https://pokeapi.co/api/v2/pokemon/2/
  3                   venusaur            https://pokeapi.co/api/v2/pokemon/3/
  4                   charmander          https://pokeapi.co/api/v2/pokemon/4/
  5                   charmeleon          https://pokeapi.co/api/v2/pokemon/5/

  Total disponibles : 1350
